In [3]:
import pandas as pd
dataset = pd.read_csv("Social_Network_Ads.csv")
dataset = pd.get_dummies(dataset, dtype=int, drop_first=True)
dataset = dataset.drop(['User ID'],axis=1)
dataset.columns

#independent, dependent variable
independent = dataset[['Age', 'EstimatedSalary', 'Gender_Male']]
dependent = dataset[['Purchased']]

#split training and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(independent, dependent, test_size=0.30, random_state=0)

#preprocessing the training and test set
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
X_train = sc.fit_transform(X_train)
X_test= sc.transform(X_test)

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
param_grid = {
                'penalty':['l1', 'l2', 'elasticnet', None],
                'solver':['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']
             }
grid = GridSearchCV(LogisticRegression(), param_grid, refit = True, verbose=3, n_jobs=-1, scoring='f1_weighted')
grid.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
55 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py", line 1172, in fit
    solver = _

GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'penalty': ['l1', 'l2', 'elasticnet', None],
                         'solver': ['lbfgs', 'liblinear', 'newton-cg',
                                    'newton-cholesky', 'sag', 'saga']},
             scoring='f1_weighted', verbose=3)

In [6]:
grid_pred = grid.predict(X_test)

In [7]:
from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(y_test, grid_pred)

In [8]:
from sklearn.metrics import classification_report
cf_report = classification_report(y_test, grid_pred)

In [9]:
print(cf_matrix)
print(cf_report)

[[74  5]
 [ 8 33]]
              precision    recall  f1-score   support

           0       0.90      0.94      0.92        79
           1       0.87      0.80      0.84        41

    accuracy                           0.89       120
   macro avg       0.89      0.87      0.88       120
weighted avg       0.89      0.89      0.89       120



In [10]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, grid.predict_proba(X_test)[:,1])

0.9493670886075949

In [11]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_pred,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'penalty': 'l1', 'solver': 'liblinear'}: 0.8906190214115365


In [12]:
re=grid.cv_results_
table=pd.DataFrame.from_dict(re)

In [13]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002235,0.002998,0.000000,0.000000,l1,lbfgs,"{'penalty': 'l1', 'solver': 'lbfgs'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,14
1,0.013509,0.003278,0.018353,0.002081,l1,liblinear,"{'penalty': 'l1', 'solver': 'liblinear'}",0.835985,0.802399,0.644599,0.927778,0.909115,0.823975,0.100806,1
2,0.000806,0.001611,0.000000,0.000000,l1,newton-cg,"{'penalty': 'l1', 'solver': 'newton-cg'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,14
3,0.003916,0.004984,0.000000,0.000000,l1,newton-cholesky,"{'penalty': 'l1', 'solver': 'newton-cholesky'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,14
4,0.002138,0.003567,0.000000,0.000000,l1,sag,"{'penalty': 'l1', 'solver': 'sag'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,14
5,0.012841,0.003849,0.014628,0.001480,l1,saga,"{'penalty': 'l1', 'solver': 'saga'}",0.835985,0.802399,0.644599,0.927778,0.909115,0.823975,0.100806,1
6,0.014190,0.005394,0.012537,0.002683,l2,lbfgs,"{'penalty': 'l2', 'solver': 'lbfgs'}",0.835985,0.802399,0.644599,0.927778,0.890114,0.820175,0.097839,3
7,0.007621,0.004209,0.012641,0.001241,l2,liblinear,"{'penalty': 'l2', 'solver': 'liblinear'}",0.835985,0.802399,0.644599,0.927778,0.890114,0.820175,0.097839,3
8,0.009841,0.002979,0.020654,0.000744,l2,newton-cg,"{'penalty': 'l2', 'solver': 'newton-cg'}",0.835985,0.802399,0.644599,0.927778,0.890114,0.820175,0.097839,3
9,0.095996,0.004783,0.010255,0.001837,l2,newton-cholesky,"{'penalty': 'l2', 'solver': 'newton-cholesky'}",0.835985,0.802399,0.644599,0.927778,0.890114,0.820175,0.097839,3


In [14]:
age_input=float(input("Age:"))
bmi_input=float(input("BMI:"))
children_input=float(input("Children:"))
sex_male_input=int(input("Sex Male 0 or 1:"))
smoker_yes_input=int(input("Smoker Yes 0 or 1:"))

Age: 25
BMI: 43
Children: 2
Sex Male 0 or 1: 1
Smoker Yes 0 or 1: 1


In [25]:
Future_Prediction=grid.predict([[age_input,bmi_input,children_input]])# change the paramter,play with it.
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]
